<a href="https://colab.research.google.com/github/russpv/SafeDrug/blob/main/GAMENet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

! pip install memory_profiler

Sun May  8 19:18:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Args

In [2]:
import argparse
def arg_parser():
    """ Parse command line arguments

    Outputs:
        arguments {object} -- object containing command line arguments
    """

    # Initializer
    parser = argparse.ArgumentParser()

    # Add arguments here
    parser.add_argument('--Test', action='store_true', default=False, help="test mode")
    parser.add_argument('--model_name', type=str, default='none', help="model name")
    parser.add_argument('--resume_path', type=str, default='none', help='resume path')
    parser.add_argument('--lr', type=float, default=5e-4, help='learning rate')
    parser.add_argument('--ddi_loss', action='store_true', default=True, help="using ddi loss")
    parser.add_argument('--target_ddi', type=float, default=0.06, help="target ddi")
    parser.add_argument('--T', type=float, default=2.0, help='T')
    parser.add_argument('--decay_weight', type=float, default=0.85, help="decay weight")
    parser.add_argument('--dim', type=int, default=64, help='dimension')
    parser.add_argument('--cuda', type=int, default=0, help='which cuda') ###
    parser.add_argument('--beta', type=int, default=0, help='SafeDrug PID loss') ###

    parser.add_argument('--smalldata', type=int, default=1, help='which cuda') ###
    parser.add_argument('--mydata', type=int, default=1, help='which cuda') ###
    parser.add_argument('--Inf_time', type=int, default=0, help='which cuda') ###
 
    # Parse and return arguments
    return(parser.parse_args(args=[]))

args = arg_parser()

In [20]:
import os
import dill
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import pandas as pd
import sys
import time
import statistics
import datetime as dt
import logging

# set seed
seed = 1203 #1203
random.seed(seed)
np.random.seed(seed) #2048
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# define data path
DATA_PATH = "drive/MyDrive/DL4H/Project/PaperCode/processed_orig/"
MYDATA_PATH = "drive/MyDrive/DL4H/Project/SAFEDRUG_lib/data/processed/"
WORKING_PATH = "drive/MyDrive/DL4H/Project/GAMENet/"
TEST_PATH = "drive/MyDrive/DL4H/Project/GAMENet/results/"

# define dataset
args.mydata = 0
args.smalldata = 0
EPOCH = 50

# define routine
args.Test = False
args.Inf_time = True

# setting
args.model_name = 'GAMENet'

args.resume_path = WORKING_PATH + 'saved/' + 'GAMENetEpoch_49_TARGET_0.06_JA_0.7602_DDI_0.05919_2022-05-08 21:32:32.705229.model'
#GAMENetEpoch_4_TARGET_0.06_JA_0.2347_DDI_0.1771_2022-05-08 18:42:56.054428.model
logger = logging.getLogger('')
logger.setLevel(logging.WARNING)

# Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Data switch
if args.mydata == 1:
    data_path = MYDATA_PATH + 'ehr.pkl'
    voc_path = MYDATA_PATH + 'vocabs.pkl'

    ehr_adj_path = MYDATA_PATH + 'ehradj.pkl'
    ddi_adj_path = MYDATA_PATH + 'ddiadj.pkl'
    ddi_mask_path = MYDATA_PATH + 'hmask.pkl'
    molecule_path = MYDATA_PATH + 'atc2SMILES.pkl'

    voc = dill.load(open(voc_path, 'rb'))
    diag_voc, pro_voc, med_voc = voc['diag_vocab'].index2word, voc['pro_vocab'].index2word, voc['med_vocab'].index2word

else:
    data_path = DATA_PATH + 'records_final.pkl'
    voc_path = DATA_PATH + 'voc_final.pkl'


    ehr_adj_path = DATA_PATH + 'ehr_adj_final.pkl'
    ddi_adj_path = DATA_PATH + 'ddi_A_final.pkl'
    ddi_mask_path = DATA_PATH + 'ddi_mask_H.pkl'
    molecule_path = DATA_PATH + 'atc3toSMILES.pkl'
    
    voc = dill.load(open(voc_path, 'rb'))
    diag_voc, pro_voc, med_voc = voc['diag_voc'].idx2word, voc['pro_voc'].idx2word, voc['med_voc'].idx2word

ehr_adj = dill.load(open(ehr_adj_path, 'rb'))
ddi_adj = dill.load(open(ddi_adj_path, 'rb'))
ddi_mask_H = dill.load(open(ddi_mask_path, 'rb'))
data = dill.load(open(data_path, 'rb'))
molecule = dill.load(open(molecule_path, 'rb')) 

if args.smalldata == 1:
    data_train = data[:200] 
    data_test = data[200:250]
    data_eval = data[250:300]
else:
    split_point = int(len(data) * 2 / 3)
    data_train = data[:split_point]
    eval_len = int(len(data[split_point:]) / 2)
    data_test = data[split_point:split_point + eval_len]
    data_eval = data[split_point+eval_len:]

# Utils

In [6]:
from sklearn.metrics import jaccard_score, roc_auc_score, precision_score, f1_score, average_precision_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import warnings
import dill
from collections import Counter
from collections import defaultdict
import torch
warnings.filterwarnings('ignore')

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

# use the same metric from DMNC
def llprint(message):
    sys.stdout.write(message)
    sys.stdout.flush()

def transform_split(X, Y):
    x_train, x_eval, y_train, y_eval = train_test_split(X, Y, train_size=2/3, random_state=1203)
    x_eval, x_test, y_eval, y_test = train_test_split(x_eval, y_eval, test_size=0.5, random_state=1203)
    return x_train, x_eval, x_test, y_train, y_eval, y_test

def sequence_output_process(output_logits, filter_token):
    pind = np.argsort(output_logits, axis=-1)[:, ::-1]

    out_list = []
    break_flag = False
    for i in range(len(pind)):
        if break_flag:
            break
        for j in range(pind.shape[1]):
            label = pind[i][j]
            if label in filter_token:
                break_flag = True
                break
            if label not in out_list:
                out_list.append(label)
                break
    y_pred_prob_tmp = []
    for idx, item in enumerate(out_list):
        y_pred_prob_tmp.append(output_logits[idx, item])
    sorted_predict = [x for _, x in sorted(zip(y_pred_prob_tmp, out_list), reverse=True)]
    return out_list, sorted_predict


def sequence_metric(y_gt, y_pred, y_prob, y_label):
    def average_prc(y_gt, y_label):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b]==1)[0]
            out_list = y_label[b]
            inter = set(out_list) & set(target)
            prc_score = 0 if len(out_list) == 0 else len(inter) / len(out_list)
            score.append(prc_score)
        return score


    def average_recall(y_gt, y_label):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = y_label[b]
            inter = set(out_list) & set(target)
            recall_score = 0 if len(target) == 0 else len(inter) / len(target)
            score.append(recall_score)
        return score


    def average_f1(average_prc, average_recall):
        score = []
        for idx in range(len(average_prc)):
            if (average_prc[idx] + average_recall[idx]) == 0:
                score.append(0)
            else:
                score.append(2*average_prc[idx]*average_recall[idx] / (average_prc[idx] + average_recall[idx]))
        return score


    def jaccard(y_gt, y_label):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = y_label[b]
            inter = set(out_list) & set(target)
            union = set(out_list) | set(target)
            jaccard_score = 0 if union == 0 else len(inter) / len(union)
            score.append(jaccard_score)
        return np.mean(score)

    def f1(y_gt, y_pred):
        all_micro = []
        for b in range(y_gt.shape[0]):
            all_micro.append(f1_score(y_gt[b], y_pred[b], average='macro'))
        return np.mean(all_micro)

    def roc_auc(y_gt, y_pred_prob):
        all_micro = []
        for b in range(len(y_gt)):
            all_micro.append(roc_auc_score(y_gt[b], y_pred_prob[b], average='macro'))
        return np.mean(all_micro)

    def precision_auc(y_gt, y_prob):
        all_micro = []
        for b in range(len(y_gt)):
            all_micro.append(average_precision_score(y_gt[b], y_prob[b], average='macro'))
        return np.mean(all_micro)

    def precision_at_k(y_gt, y_prob_label, k):
        precision = 0
        for i in range(len(y_gt)):
            TP = 0
            for j in y_prob_label[i][:k]:
                if y_gt[i, j] == 1:
                    TP += 1
            precision += TP / k
        return precision / len(y_gt)
    try:
        auc = roc_auc(y_gt, y_prob)
    except ValueError:
        auc = 0
    p_1 = precision_at_k(y_gt, y_label, k=1)
    p_3 = precision_at_k(y_gt, y_label, k=3)
    p_5 = precision_at_k(y_gt, y_label, k=5)
    f1 = f1(y_gt, y_pred)
    prauc = precision_auc(y_gt, y_prob)
    ja = jaccard(y_gt, y_label)
    avg_prc = average_prc(y_gt, y_label)
    avg_recall = average_recall(y_gt, y_label)
    avg_f1 = average_f1(avg_prc, avg_recall)

    return ja, prauc, np.mean(avg_prc), np.mean(avg_recall), np.mean(avg_f1)


def multi_label_metric(y_gt, y_pred, y_prob):

    def jaccard(y_gt, y_pred):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = np.where(y_pred[b] == 1)[0]
            inter = set(out_list) & set(target)
            union = set(out_list) | set(target)
            jaccard_score = 0 if union == 0 else len(inter) / len(union)
            score.append(jaccard_score)
        return np.mean(score)

    def average_prc(y_gt, y_pred):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = np.where(y_pred[b] == 1)[0]
            inter = set(out_list) & set(target)
            prc_score = 0 if len(out_list) == 0 else len(inter) / len(out_list)
            score.append(prc_score)
        return score

    def average_recall(y_gt, y_pred):
        score = []
        for b in range(y_gt.shape[0]):
            target = np.where(y_gt[b] == 1)[0]
            out_list = np.where(y_pred[b] == 1)[0]
            inter = set(out_list) & set(target)
            recall_score = 0 if len(target) == 0 else len(inter) / len(target)
            score.append(recall_score)
        return score

    def average_f1(average_prc, average_recall):
        score = []
        for idx in range(len(average_prc)):
            if average_prc[idx] + average_recall[idx] == 0:
                score.append(0)
            else:
                score.append(2*average_prc[idx]*average_recall[idx] / (average_prc[idx] + average_recall[idx]))
        return score

    def f1(y_gt, y_pred):
        all_micro = []
        for b in range(y_gt.shape[0]):
            all_micro.append(f1_score(y_gt[b], y_pred[b], average='macro'))
        return np.mean(all_micro)

    def roc_auc(y_gt, y_prob):
        all_micro = []
        for b in range(len(y_gt)):
            all_micro.append(roc_auc_score(y_gt[b], y_prob[b], average='macro'))
        return np.mean(all_micro)

    def precision_auc(y_gt, y_prob):
        all_micro = []
        for b in range(len(y_gt)):
            all_micro.append(average_precision_score(y_gt[b], y_prob[b], average='macro'))
        return np.mean(all_micro)

    def precision_at_k(y_gt, y_prob, k=3):
        precision = 0
        sort_index = np.argsort(y_prob, axis=-1)[:, ::-1][:, :k]
        for i in range(len(y_gt)):
            TP = 0
            for j in range(len(sort_index[i])):
                if y_gt[i, sort_index[i, j]] == 1:
                    TP += 1
            precision += TP / len(sort_index[i])
        return precision / len(y_gt)

    # roc_auc
    try:
        auc = roc_auc(y_gt, y_prob)
    except:
        auc = 0
    # precision
    p_1 = precision_at_k(y_gt, y_prob, k=1)
    p_3 = precision_at_k(y_gt, y_prob, k=3)
    p_5 = precision_at_k(y_gt, y_prob, k=5)
    # macro f1
    f1 = f1(y_gt, y_pred)
    # precision
    prauc = precision_auc(y_gt, y_prob)
    # jaccard
    ja = jaccard(y_gt, y_pred)
    # pre, recall, f1
    avg_prc = average_prc(y_gt, y_pred)
    avg_recall = average_recall(y_gt, y_pred)
    avg_f1 = average_f1(avg_prc, avg_recall)

    return ja, prauc, np.mean(avg_prc), np.mean(avg_recall), np.mean(avg_f1)

def ddi_rate_score(record, path=ddi_adj_path): ###
    # ddi rate
    ddi_A = dill.load(open(path, 'rb'))
    all_cnt = 0
    dd_cnt = 0
    for patient in record:
        for adm in patient:
            med_code_set = adm
            for i, med_i in enumerate(med_code_set):
                for j, med_j in enumerate(med_code_set):
                    if j <= i:
                        continue
                    all_cnt += 1
                    if ddi_A[med_i, med_j] == 1 or ddi_A[med_j, med_i] == 1:
                        dd_cnt += 1
    if all_cnt == 0:
        return 0
    return dd_cnt / all_cnt


def create_atoms(mol, atom_dict):
    """Transform the atom types in a molecule (e.g., H, C, and O)
    into the indices (e.g., H=0, C=1, and O=2).
    Note that each atom index considers the aromaticity.
    """
    atoms = [a.GetSymbol() for a in mol.GetAtoms()]
    for a in mol.GetAromaticAtoms():
        i = a.GetIdx()
        atoms[i] = (atoms[i], 'aromatic')
    atoms = [atom_dict[a] for a in atoms]
    return np.array(atoms)

def create_ijbonddict(mol, bond_dict):
    """Create a dictionary, in which each key is a node ID
    and each value is the tuples of its neighboring node
    and chemical bond (e.g., single and double) IDs.
    """
    i_jbond_dict = defaultdict(lambda: [])
    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bond = bond_dict[str(b.GetBondType())]
        i_jbond_dict[i].append((j, bond))
        i_jbond_dict[j].append((i, bond))
    return i_jbond_dict

def extract_fingerprints(radius, atoms, i_jbond_dict,
                         fingerprint_dict, edge_dict):
    """Extract the fingerprints from a molecular graph
    based on Weisfeiler-Lehman algorithm.
    """

    if (len(atoms) == 1) or (radius == 0):
        nodes = [fingerprint_dict[a] for a in atoms]

    else:
        nodes = atoms
        i_jedge_dict = i_jbond_dict

        for _ in range(radius):

            """Update each node ID considering its neighboring nodes and edges.
            The updated node IDs are the fingerprint IDs.
            """
            nodes_ = []
            for i, j_edge in i_jedge_dict.items():
                neighbors = [(nodes[j], edge) for j, edge in j_edge]
                fingerprint = (nodes[i], tuple(sorted(neighbors)))
                nodes_.append(fingerprint_dict[fingerprint])

            """Also update each edge ID considering
            its two nodes on both sides.
            """
            i_jedge_dict_ = defaultdict(lambda: [])
            for i, j_edge in i_jedge_dict.items():
                for j, edge in j_edge:
                    both_side = tuple(sorted((nodes[i], nodes[j])))
                    edge = edge_dict[(both_side, edge)]
                    i_jedge_dict_[i].append((j, edge))

            nodes = nodes_
            i_jedge_dict = i_jedge_dict_

    return np.array(nodes)


def buildMPNN(molecule, med_voc, radius=1, device="cpu:0"):

    atom_dict = defaultdict(lambda: len(atom_dict))
    bond_dict = defaultdict(lambda: len(bond_dict))
    fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
    edge_dict = defaultdict(lambda: len(edge_dict))
    MPNNSet, average_index = [], []

    for index, atc3 in med_voc.items():

        smilesList = list(molecule[atc3])
        """Create each data with the above defined functions."""
        counter = 0 # counter how many drugs are under that ATC-3
        for smiles in smilesList:
            try:
                mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
                atoms = create_atoms(mol, atom_dict)
                molecular_size = len(atoms)
                i_jbond_dict = create_ijbonddict(mol, bond_dict)
                fingerprints = extract_fingerprints(radius, atoms, i_jbond_dict,
                                                    fingerprint_dict, edge_dict)
                adjacency = Chem.GetAdjacencyMatrix(mol)
                # if fingerprints.shape[0] == adjacency.shape[0]:
                for _ in range(adjacency.shape[0] - fingerprints.shape[0]):
                    fingerprints = np.append(fingerprints, 1)
                
                fingerprints = torch.LongTensor(fingerprints).to(device)
                adjacency = torch.FloatTensor(adjacency).to(device)
                MPNNSet.append((fingerprints, adjacency, molecular_size))
                counter += 1
            except:
                continue
        
        average_index.append(counter)

        """Transform the above each data of numpy
        to pytorch tensor on a device (i.e., CPU or GPU).
        """

    N_fingerprint = len(fingerprint_dict)
    # transform into projection matrix
    n_col = sum(average_index)
    n_row = len(average_index)

    average_projection = np.zeros((n_row, n_col))
    col_counter = 0
    for i, item in enumerate(average_index):
        if item > 0:
            average_projection[i, col_counter : col_counter + item] = 1 / item
        col_counter += item

    return MPNNSet, N_fingerprint, torch.FloatTensor(average_projection)

# Model

In [7]:
class RNN(nn.Module):
    def __init__(self, vocab_size, emb_dim=16, device=torch.device('cpu:0')):
        super(RNN, self).__init__()
        """
        FUNCTIONALITY:
            1. embed the input medical code
            2. take the average for one visit
            3. pass to the RNN visit by visit
            4. output embedding seq for each visit
        """
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=emb_dim)
        self.rnn = nn.GRU(input_size=emb_dim, hidden_size=emb_dim, bias=True, batch_first=True)
        self.init_weights()
        self.device = device
    
    def forward(self, codes):
        """
        Input:
            codes: [[codes for visit1], [codes for visit2], ...] for ONE patient only
                - e.g., [[0,1], [1,2]]  
        Output:
            embedding seq
                - dim: (#visits, emb_dim) 
        """
        emb_list = []
        for code in codes:
            codelist = torch.LongTensor(code).to(self.device) # token indices for retrieving their embeddings
            emb = self.embeddings(codelist) # get embed_dim-deep latent representation per token

            # take the mean and make one sample per batch
            emb_mean = emb.mean(dim=0).unsqueeze(dim=0) #collapse into one embedding per visit CHECK THIS, MAY NOT NEED BATCH DIM
            emb_list.append(emb_mean)
            
        emb_seq = torch.cat(emb_list, dim=0).unsqueeze(dim=0) # CHECK THIS, MAY NOT NEED BATCH DIM or is this to rep (visit, codetype, code)
        result, _ = self.rnn(emb_seq) # take output, (batch, num_visits, embed_dim) but we ain't batching?
        
        logger.debug(f'RNN output size: {result.size()}')
        return result
    
    def init_weights(self):
        torch.nn.init.normal_(self.embeddings.weight)
        for param in self.rnn.parameters():
            if len(param.shape) >= 2:
                torch.nn.init.orthogonal_(param.data)
            else:
                torch.nn.init.normal_(param.data)

In [8]:
class PatientQuery(nn.Module):
    def __init__(self, vocab_diag, vocab_prod, emb_dim=16, device=torch.device('cpu:0')):
        super(PatientQuery, self).__init__()
        """
        FUNCTIONALITY:
            1. input the embedding given by RNN
            2. output the patient representation
        """
        self.rnn_diag = RNN(vocab_diag, emb_dim, device)
        self.rnn_prod = RNN(vocab_prod, emb_dim, device)
        self.linear = nn.Linear(in_features=emb_dim*2, out_features=emb_dim)
        
    def forward(self, codes_diag, codes_prod):
        """
        Input:
            codes_diag: [[diag codes for visit1], [diag codes for visit2], ...]
                - e.g., [[0,1], [1,2]]  
            codes_prod: [[prod codes for visit1], [prod codes for visit2], ...]
                - e.g., [[0,1], [1,2]]  
        output:
            query embedding:
                - size: (#visits, emd_dim)
        """
        emb_diag = self.rnn_diag(codes_diag)
        emb_prod = self.rnn_prod(codes_prod)
        emb_cat = torch.cat((emb_diag, emb_prod), dim=-1) 
        
        result = self.linear(emb_cat) 
        logger.debug(f'PatientQuery linear output size: {result.size()}')
        return result.squeeze(dim=0)

In [9]:
class GraphConvolution(nn.Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.mm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'
    
class GCN(nn.Module):
    def __init__(self, vocab_med, adj, emb_dim=16, device=torch.device('cpu:0')):
        super(GCN, self).__init__()
        """
        FUNCTIONALITY: given the adjacency matrix, do graph convolution twice
        """
        
        adj = self.normalize(adj + np.eye(adj.shape[0])) # A-tilde
        self.adj = torch.FloatTensor(adj).to(device) # A-tilde
        
        self.gcn1 = GraphConvolution(in_features=vocab_med, out_features=emb_dim)
        self.gcn2 = GraphConvolution(in_features=emb_dim, out_features=emb_dim)

        # the initial feature
        self.x = torch.eye(vocab_med).to(device) ## scalar argument
    
    def forward(self):

        x = self.gcn1(self.x, self.adj)
        x = F.relu(x)
        x = self.gcn2(x, self.adj)
        
        logger.debug(f'GCN output size: {x.size()}')
        return x
    
    def normalize(self, adj):
        adj = adj / (adj @ np.ones(adj.shape) + 1e-8) # WE DON'T TAKE THE ROOT?
        return adj

In [10]:
class MemoryBank(nn.Module):
    def __init__(self, vocab_med, adj_ehr, adj_ddi, emb_dim=16, device=torch.device('cpu:0')):
        super(MemoryBank, self).__init__()
        """
        FUNCTIONALITY: conbime information from EHR graph and DDI graph
        """
        self.gcn_ehr = GCN(vocab_med, adj_ehr, emb_dim, device)
        self.gcn_ddi = GCN(vocab_med, adj_ddi, emb_dim, device)
        
        self.weight = nn.Parameter(torch.FloatTensor(1))
        self.weight.data.uniform_(-0.1, 0.1)
        
    def forward(self):        

        info_ehr = self.gcn_ehr.forward()
        info_ddi = self.gcn_ddi.forward()

        info_comb = info_ehr - info_ddi * self.weight
        logger.debug(f'MB output info_comb size: {info_comb.size()}')
        return info_comb

In [11]:
class DynamicMemory(nn.Module):
    def __init__(self, vocab_med, device):
        super(DynamicMemory, self).__init__()
        """
        FUNCTIONALITY: generate a historical mapping: query embedding -> multi-hot medication vector
        """
        self.vocab_med = vocab_med # Scalar
        self.device = device
    
    def forward(self, queries, codes_med):
        """
        Input:  queries
                    - this is the historical query embedding, given by PatientQuery Module
                    - size: (#visits - 1, emb_dim), delete the current query
                codes_med
                    - this is the historical groud truth med vector
                    - format: a list of length (#visits - 1)
        """
        #queries = queries.squeeze(dim=0) # for some reason we need to remove the batch/patient dim
        '''HERE remove the last list from within each list of the sequence, dim0 is preserved
        '''
        DM_key = queries # patient embeddings, the last visit is deleted upstream if at all
        DM_value = np.zeros((queries.shape[0], self.vocab_med)) # multi-hot medication vector

        for i, _ in enumerate(DM_value): # per visit
            if i < len(codes_med): # if meds exist for the visit
                for code in codes_med[i]:  # meds of the visit
                    DM_value[i][code] = 1
        
        DM_value = torch.FloatTensor(DM_value).to(self.device)
        logger.debug(f'DM_key (visit, embed) size: {DM_key.size()} DM_value (visit, med) size: {DM_value.size()}')
        return DM_key, DM_value

In [12]:
class Fact1(nn.Module):
    def __init__(self, queries):
        super(Fact1, self).__init__()
        """
        FUNCTIONALITY: extract the final embedding from input queries, q^t
        """
        self.queries = queries
    
    def forward(self):
        result = self.queries[-1].unsqueeze(dim=0)
        logging.debug(f'Fact1 (1, embed): {result.size()}')
        return result
    

class Fact2(nn.Module):
    def __init__(self, query, MB):
        super(Fact2, self).__init__()
        """
        FUNCTIONALITY: get attention information from MB, o^t_b
        Input:
            query
                - this is the last embedding
            MB
                - is the memory bank
        """
        self.query = query
        self.MB = MB
    
    def forward(self):
        attn_score = torch.mm(self.query, self.MB.t()) # (1, 16) x (16, 131)
        attn_matrix = torch.softmax(attn_score, dim=-1) # (1, 131)
        result = torch.mm(attn_matrix, self.MB) # (1, 131) x (131, 128)

        logger.debug(f'Fact2 (1, embed): {result.size()}')
        return result
    
class Fact3(nn.Module):
    def __init__(self, query, MB, DM_key, DM_value):
        super(Fact3, self).__init__()
        """
        FUNCTIONALITY: similar to Fact2, get information from the DM, o_d^t
        """
        self.query = query
        self.MB = MB
        self.DM_key = DM_key
        self.DM_value = DM_value
    
    def forward(self):
        attn = torch.softmax(torch.mm(self.query, self.DM_key.t()), dim=1) # (1, 16) x (16, visits)
        value = torch.mm(attn, self.DM_value) # (1, visits) x (visits, 131)
        out = torch.mm(value, self.MB) # (1, 131) x (131, 16)

        logger.debug(f'Fact3 (1, embed): {out.size()}')
        return out # (1, 16)

In [13]:
class OutNet(nn.Module):
    def __init__(self, vocab_med, emb_dim=16):
        super(OutNet, self).__init__()
        """
        FUNCTIONALITY: combine fact1, fact2, fact3 to do final prediction
        """
        self.fc1 = nn.Linear(in_features=3*emb_dim, out_features=2*emb_dim, bias=True)
        self.fc2 = nn.Linear(in_features=2*emb_dim, out_features=vocab_med, bias=True)
        
    def forward(self, fact1, fact2, fact3):
        """
        Input:
            fact1, fact2, fact3:
                - three facts q^t, o^t_b, o^t_d
                - each size: (1, emb_dim)
        """
        memory_out = torch.cat((fact1, fact2, fact3), dim=-1)
        x = self.fc1(memory_out)
        x = torch.relu(x)
        result = self.fc2(x)

        logger.debug(f'OutNet (1, med) size:{result.size()}')
        return result # (1, 131)

In [14]:
class GAMENet(nn.Module):
    def __init__(self, vocab_size, adj_ehr, adj_ddi, emb_dim=64, device=torch.device('cpu:0'), ddi_in_memory=True):
        super(GAMENet, self).__init__()

        vocab_diag, vocab_prod, vocab_med = vocab_size[0], vocab_size[1], vocab_size[2]
        self.patient = PatientQuery(vocab_diag, vocab_prod, emb_dim, device)
        self.memorybank = MemoryBank(vocab_med, adj_ehr, adj_ddi, emb_dim, device)
        self.dynamicmemory = DynamicMemory(vocab_med, device)
        self.outnet = OutNet(vocab_med, emb_dim)
        self.tensor_ddi_adj = torch.FloatTensor(ddi_adj).to(device)

    def forward(self, x):
        """
        input:
                - a list of length #visits / for med -1
                - each element is also itself a list
        """
        codes_diag = [visit[0] for visit in x]
        codes_prod = [visit[1] for visit in x]
        codes_med = [visit[2] for visit in x] 

        queries = self.patient(codes_diag, codes_prod)
        MB = self.memorybank()
        DM_key, DM_value = self.dynamicmemory(queries, codes_med) # already squeeze downstream
        
        # extract three memory outputs, assign to fact1, fact2, fact3
        fact1 = Fact1(queries)()
        fact2 = Fact2(fact1, MB)()
        fact3 = Fact3(fact1, MB, DM_key, DM_value)()

        result = self.outnet(fact1, fact2, fact3)
        logger.debug(f'GAMENet shape: {result.size()} type: {type(result)}')
        if self.training:
            neg_pred_prob = torch.sigmoid(result)
            neg_pred_prob = neg_pred_prob.t() * neg_pred_prob  # (voc_size, voc_size)
            batch_neg = 0.0005 * neg_pred_prob.mul(self.tensor_ddi_adj).sum()
            return result, batch_neg
        else:
            return result

# Training


In [15]:
import dill
import numpy as np
import argparse
from collections import defaultdict
from sklearn.metrics import jaccard_score
from torch.optim import Adam
import os
import torch
import time

import torch.nn.functional as F

# evaluate
def eval(model, data_eval, voc_size, epoch):
    model.eval()

    smm_record = []
    ja, prauc, avg_p, avg_r, avg_f1 = [[] for _ in range(5)]
    med_cnt, visit_cnt = 0, 0

    for step, input in enumerate(data_eval):
        y_gt, y_pred, y_pred_prob, y_pred_label = [], [], [], []
        for adm_idx, adm in enumerate(input):
            target_output = model(input[:adm_idx+1])

            y_gt_tmp = np.zeros(voc_size[2])
            y_gt_tmp[adm[2]] = 1
            y_gt.append(y_gt_tmp)

            # prediction prod
            target_output = torch.sigmoid(target_output).detach().cpu().numpy()[0]
            y_pred_prob.append(target_output)
            
            # prediction med set
            y_pred_tmp = target_output.copy()
            y_pred_tmp[y_pred_tmp>=0.5] = 1
            y_pred_tmp[y_pred_tmp<0.5] = 0
            y_pred.append(y_pred_tmp)

            # prediction label
            y_pred_label_tmp = np.where(y_pred_tmp == 1)[0]
            y_pred_label.append(sorted(y_pred_label_tmp))
            visit_cnt += 1
            med_cnt += len(y_pred_label_tmp)

        smm_record.append(y_pred_label)
        adm_ja, adm_prauc, adm_avg_p, adm_avg_r, adm_avg_f1 = multi_label_metric(np.array(y_gt), np.array(y_pred), np.array(y_pred_prob))

        ja.append(adm_ja)
        prauc.append(adm_prauc)
        avg_p.append(adm_avg_p)
        avg_r.append(adm_avg_r)
        avg_f1.append(adm_avg_f1)
        llprint('\rtest step: {} / {}'.format(step+1, len(data_eval)))

    # ddi rate
    ddi_rate = ddi_rate_score(smm_record, path=ddi_adj_path) ###

    llprint('\nDDI Rate: {:.4}, Jaccard: {:.4},  PRAUC: {:.4}, AVG_PRC: {:.4}, AVG_RECALL: {:.4}, AVG_F1: {:.4}, AVG_MED: {:.4}\n'.format(
        ddi_rate, np.mean(ja), np.mean(prauc), np.mean(avg_p), np.mean(avg_r), np.mean(avg_f1), med_cnt / visit_cnt
    ))

    return ddi_rate, np.mean(ja), np.mean(prauc), np.mean(avg_p), np.mean(avg_r), np.mean(avg_f1), med_cnt / visit_cnt

def main():

    device = torch.device(f'cuda:{args.cuda}' if torch.cuda.is_available() else 'cpu')

    voc_size = (len(diag_voc), len(pro_voc), len(med_voc))
    model = GAMENet(voc_size, ehr_adj, ddi_adj, device=device)
    #model.load_state_dict(torch.load(open(args.resume_path, 'rb'))) # switch if resuming

    if args.Inf_time:
        #https://towardsdatascience.com/the-correct-way-to-measure-inference-time-of-deep-neural-networks-304a54e5187f
        model.load_state_dict(torch.load(open(args.resume_path, 'rb'))) 
        model.to(device=device)
        tic = time.time()

        starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
        repetitions = len(data_test)
        timings = np.zeros((repetitions,1))
        dummy_input = [[[13, 98, 585, 1065, 21, 37, 454, 278], [69, 47], [4, 22, 12, 2, 67, 0, 86]],\
                       [[377, 326, 21, 46, 454], [115, 94], [3, 6, 12, 14, 5, 22, 2, 29, 1, 16, 11, 86]],\
                       [[377, 246, 453, 46, 21, 454], [151, 127, 128], [14, 2, 6, 29, 18, 0, 86]], [[963, 258, 32, 93, 94, 13, 103, 571, 21], [164, 423, 424, 425, 95, 426, 361, 48, 46, 2], [5, 4, 6, 7, 9, 11, 12, 3, 13, 16, 14, 22, 1, 2, 29, 44, 45, 48, 56, 20, 76, 86]]]

        #GPU-WARM-UP
        for _ in range(10):
            _ = model(dummy_input)
        count = 0

        # MEASURE PERFORMANCE
        with torch.no_grad():
            #for rep in range(repetitions):
            for rep, example in enumerate(data_test):
                starter.record()
                _ = model(example)
                ender.record()
                # WAIT FOR GPU SYNC
                torch.cuda.synchronize()
                curr_time = starter.elapsed_time(ender)
                timings[rep] = curr_time
                count += 1

        mean_syn = np.sum(timings) / repetitions
        std_syn = np.std(timings)
        print(f'Inference reps {count}, average: {mean_syn} \u00B1 {std_syn} seconds')

        data = np.array([mean_syn, std_syn, count])
        df = pd.DataFrame(data, index=['mean inference time', 'stdev', 'reps'])
        df.to_csv(TEST_PATH + 'Inf_' + args.model_name + device.type + f'{dt.datetime.now()}' + '.csv' )

        return

    if args.Test:
        model.load_state_dict(torch.load(open(args.resume_path, 'rb')))
        model.to(device=device)
        tic = time.time()

        ddi_list, ja_list, prauc_list, f1_list, med_list = [], [], [], [], []

        result = []
        for _ in range(10):
            time_start = time.time()
            test_sample = np.random.choice(data_test, round(len(data_test) * 0.8), replace=True)
            ddi_rate, ja, prauc, avg_p, avg_r, avg_f1, avg_med = eval(model, test_sample, voc_size, 0)
            time_sample = time.time() - time_start ###
            result.append([ddi_rate, ja, avg_f1, prauc, avg_med, time_sample])
            
        result = np.array(result)
        mean = result.mean(axis=0)
        std = result.std(axis=0)

        outstring = ""
        for m, s in zip(mean, std):
            outstring += "{:.4f} "u"\u00B1"" {:.4f} & ".format(m, s) ###

        print(outstring)
        time_round = time.time() - tic
        print(f'test time: {time_round}')
        
        elapsed_time = [0. for _ in range(5)]
        elapsed_time.append(time_round)
        data = np.array([mean, std, elapsed_time])

        df = pd.DataFrame(data, columns=['ddi', 'ja', 'prauc', 'f1', 'med', 'time'], index=['mean', 'std', 'seconds'])
        df.to_csv(TEST_PATH + 'Test_' + args.model_name + device.type + f'{dt.datetime.now()}' + '.csv' )

        return 

    if 'cpu' not in device.type:
        torch.cuda.reset_peak_memory_stats() # flush 
    model.to(device=device)
    print('parameters', sum(p.numel() for p in model.parameters() if p.requires_grad)) ###
    # exit()
    optimizer = Adam(list(model.parameters()), lr=args.lr)

    # start iterations
    history = defaultdict(list)
    best_epoch, best_ja = 0, 0

    times_train, times_eval = [], [] ###
    for epoch in range(EPOCH):
        time_start = time.time() ###
        print('\nepoch {} --------------------------'.format(epoch + 1))
        normal_loss_count, ddi_loss_count = 0, 0
        model.train()
        beta_log = [] ###
        for step, input in enumerate(data_train): 

            loss = 0
            for idx, adm in enumerate(input):

                seq_input = input[:idx+1]
                loss_bce_target = np.zeros((1, voc_size[2]))
                loss_bce_target[:, adm[2]] = 1

                loss_multi_target = np.full((1, voc_size[2]), -1)
                for idx, item in enumerate(adm[2]):
                    loss_multi_target[0][idx] = item

                result, loss_ddi = model(seq_input) # add loss_ddi

                loss_bce = F.binary_cross_entropy_with_logits(result, torch.FloatTensor(loss_bce_target).to(device))
                loss_multi = F.multilabel_margin_loss(F.sigmoid(result), torch.LongTensor(loss_multi_target).to(device))

                if args.ddi_loss:
                    result = torch.sigmoid(result).detach().cpu().numpy()[0]
                    result[result >= 0.5] = 1
                    result[result < 0.5] = 0
                    y_label = np.where(result == 1)[0] #indices where 1
                    current_ddi_rate = ddi_rate_score([[y_label]], path=ddi_adj_path)
                    if current_ddi_rate <= args.target_ddi:
                        loss = 0.9 * loss_bce + 0.01 * loss_multi
                        normal_loss_count += 1
                    elif args.beta == 0:
                        rnd = np.exp((args.target_ddi - current_ddi_rate)/args.T)
                        if np.random.rand(1) < rnd:
                            loss = loss_ddi
                            ddi_loss_count += 1
                        else:
                            loss = 0.9 * loss_bce + 0.01 * loss_multi
                            normal_loss_count += 1
                    else: 
                        beta = max(0, 1 - (current_ddi_rate - args.target_ddi) / args.kp) # per SafeDrug
                        loss = beta * (0.95 * loss_bce + 0.05 * loss_multi) + (1 - beta) * loss_ddi # alpha = 0.95
                    if args.beta == 1: beta_log.append(beta)
                else:
                    loss = 0.9 * loss_bce + 0.01 * loss_multi

                optimizer.zero_grad()
                loss.backward(retain_graph=True)
                optimizer.step()
            
            if args.beta == 0:
                llprint(f'\rtraining step: {step+1} / {len(data_train)} loss: {loss} loss_ddi: {loss_ddi} ') ###
            else:
                llprint(f'\rtraining step: {step+1} / {len(data_train)} loss: {loss} loss_ddi: {loss_ddi}  beta: {beta}') ###
        print()
        if args.beta == 1: print(f'\navg_beta: {statistics.mean(beta_log)}') ###
        time_end = time.time()  ###
        ddi_rate, ja, prauc, avg_p, avg_r, avg_f1, avg_med = eval(model, data_eval, voc_size, epoch)
        time_train = time_end - time_start ###
        time_eval = time.time() - time_end  ###
        print(f'training time: {time_train}, test time: {time_eval}') ###

        times_train.append(time_train) ###
        times_eval.append(time_eval) ###

        history['ja'].append(ja)
        history['ddi_rate'].append(ddi_rate)
        history['avg_p'].append(avg_p)
        history['avg_r'].append(avg_r)
        history['avg_f1'].append(avg_f1)
        history['prauc'].append(prauc)
        history['med'].append(avg_med)

        if epoch >= 5:
            print('ddi: {}, Med: {}, Ja: {}, F1: {}, PRAUC: {}'.format(
                np.mean(history['ddi_rate'][-5:]),
                np.mean(history['med'][-5:]),
                np.mean(history['ja'][-5:]),
                np.mean(history['avg_f1'][-5:]),
                np.mean(history['prauc'][-5:])
                ))

        torch.save(model.state_dict(), open(WORKING_PATH +''.join(('saved/', args.model_name, \
            'Epoch_{}_TARGET_{:.2}_JA_{:.4}_DDI_{:.4}_{}.model'.format(epoch, args.target_ddi, ja, ddi_rate, dt.datetime.now()))), 'wb')) ###

        if epoch != 0 and best_ja < ja:
            best_epoch = epoch
            best_ja = ja

        print('best_epoch: {}'.format(best_epoch))

    dill.dump(history, open(WORKING_PATH +'history_{}_{}.pkl'.format(args.model_name, dt.datetime.now()), 'wb')) ###
    
    timings = np.array(list(zip(times_train, times_eval))) ###
    df = pd.DataFrame(timings, columns=['train', 'test']) ###
    df.to_csv(TEST_PATH + 'TimesTrain_' + args.model_name + f'{dt.datetime.now()}' + '.csv' ) ###

    # Maximum cuda memory allocated
    if 'cpu' not in device.type:
        print(f'peak training memory allocated: {torch.cuda.max_memory_allocated(device)}')



# Execute

In [21]:
if __name__ == '__main__':
    %reload_ext memory_profiler
    %memit -r1 main()

Inference reps 1058, average: 2.6846971917467894 ± 1.3316779496684863 seconds
peak memory: 2798.84 MiB, increment: 0.25 MiB


In [17]:
!nvidia-smi

Sun May  8 21:32:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    38W / 250W |    973MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------